# Binaryzacja

### Zadanie domowe - binaryzacja adaptacyjna w oknach z interpolacją.

Pokazana w ramach podstawowej części ćwiczenia binaryzacja adaptacyjna działa dobrze, ale jest dość złożona obliczeniowo (choć oczywiście należy mieć świadomość, że implementację można zoptymalizować i wyeliminować pewne powtarzające się obliczenia).
Binaryzację adaptacyjną można również realizować w nieco innym wariancie - w oknach.
Ogólna idea jest następująca: wejściowy obraz dzielimy na nienachodzące (rozłączne) okna - wygodnie jest założyć, że są one kwadratowe i o rozmiarze będącym potęgą liczby 2.
W każdym z okien obliczamy próg - niech to będzie średnia i stosujemy binaryzację lokalną.
Jak nietrudno się domyślić efekt nie będzie dobry, gdyż na granicach okien wystąpią artefakty.
Aby je wyeliminować należy zastosować interpolację, co zostanie szczegółowo opisane poniżej.
Warto zaznaczyć, że podobny mechanizm interpolacji stosowany jest w poznanym wcześniej algorytmie CLAHE.



Na początek zaimplementujemy wariant metody bez interpolacji:
1. Wczytaj obraz _T.png_.
2. W dwóch pętlach `for`, dla okien o ustalonym wymiarze $W$ (potęga 2), oblicz średnią:
- pętle powinny mieć krok $W$,
- wynik należy zapisać w pomocniczej tablicy,

3. W kolejnych dwóch pętlach `for` (o kroku 1) przeprowadź binaryzację z wyznaczonymi progami.
   Wyświetl wyniki - czy jest on poprawny?

In [ ]:
# Obraz T.png nie jest dotępny
import numpy as np, cv2, matplotlib.pyplot as plt
catalogue = cv2.imread('catalogue.png',cv2.IMREAD_GRAYSCALE)
def bin_int(img_org, W = 8):
    img = np.copy(img_org)
    (X, Y) = img.shape
    to_bin = lambda x,y: 255 if x>=y else 0
    frame_func = np.vectorize(to_bin)
    for x in range(0,X,W):
        for y in range(0,Y,W):
            frame = img[x:x+W,y:y+W]
            mean = np.mean(frame)
            img[x:x+W,y:y+W] = frame_func(frame, mean)
    return img
plt.imshow(bin_int(catalogue,16),'gray')
plt.show()

4. Rozwiązaniem problemu artefaktów na obrazie jest zastosowanie interpolacji.
   Próg binaryzacji dla danego okna wyliczany jest na podstawie progów z sąsiednich okien.
   Możliwe są 3 przypadki:
   - piksel leży w rogach obrazu - wtedy za próg przyjmuje się wartość dla danego okna,
   - piksel leży na krawędzi obrazu - wtedy za próg przyjmuje się wartość obliczoną na podstawie dwóch okien,
   - piksel leży w środku - wtedy próg jest obliczany na podstawie 4 sąsiednich okien.
   Uwaga. Proszę to sobie "rozrysować" pamiętając o tym, że dane okno należy podzielić na cztery części (inaczej analiza nie będzie poprawna).

5. Implementujemy interpolację.
   Potrzebujemy do tego znać progi 1-4, ale dla przejrzystości obliczeń lepiej zawsze przyjąć 4 oraz odległości od rozważnego piksela do środka sąsiednich okien (też w ogólnym przypadku 4):
   - całość sprowadza się do określania pozycji piksela,
   - na początek rozważmy przypadek czterech narożników - trzeba napisać `if`, który je wyznaczy,
   - warto sprawdzić, czy nie popełniliśmy błędu i np. tymczasowo do obrazu wynikowego w tym miejscu przypisać wartość 255. Efekt powinien być taki, że widoczne będą tylko narożniki.
   - drugi przypadek do brzegi - postępujemy podobnie jak przy narożnikach, przy czym osobno wydzielamy brzegi pionowe i poziome. Tu też warto sobie obrazek "pokolorować".
   - na koniec wyznaczamy piksele w środku.
   - analizując poprawność proszę zwrócić uwagę na to, żeby nie było przerw pomiędzy obszarami.
   - mając podział możemy dla każdego z obszarów wyliczyć cztery progi ($t11, t12, t21, t22$):
        - dla narożników wartość ta będzie identyczna i wynosi po prostu `t11 =t[jT][iT]`, gdzie `iT=i//W` oraz `jT=j//W`.
          Uwaga. Proszę używać indeksów tymczasowych $jT,iT$, gdyż będą potrzebne w dalszych obliczeniach.
        - dla brzegów pionowych występują dwie wartości: okno bieżące i sąsiednie.
          Wyznaczenie współrzędnej poziomej jest proste (jak dla narożników).
          Nad współrzędną pionową trzeba się chwilę zastanowić - aby nie rozważać wielu przypadków można od bieżącej współrzędnej odjąć połowę rozmiaru okna i dopiero później wykonać dzielenie przez rozmiar okna.
          W ten sposób otrzymujemy indeks okna o mniejszej współrzędnej.
          Indeks drugiego uzyskamy dodając 1.
          Proszę się zastanowić dlaczego to działa.
        - dla brzegów poziomych należy postąpić analogicznie,
        - obliczenia dla obszaru wewnątrz powinny być już oczywiste.
   - kolejny krok to wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami.
     Przykładowo dla osi X wygląda to następująco: `dX1 = i - W/2 - iT*W` oraz `dX2 = (iT+1)*W - i-W/2`.
     Dla osi Y analogicznie.
     Ponownie proszę się zastanowić dlaczego to jest poprawne - najlepiej to sobie narysować.
   - ostatni krok to interpolacja dwuliniowa.
     Wykonamy ją w trzech krokach:
     - interpolacja w osi X dla dwóch górnych okien - sprowadza się ona do średniej ważonej pomiędzy wartościami $t11$ i $t12$, przy czym wagi to odpowiednio $dX2/W$ i $dX1/W$.
       Ponownie na podstawie rysunku proszę to przemyśleć.
     - interpolacja w osi X dla dolnych okien jest analogiczna,
     - interpolacja w osi Y również jest analogiczna, z tym, że wejściem są dwa wyniki interpolacji w poziomie.

6. "Kropka nad i" to oczywiście binaryzacja z wyznaczonym poprzez interpolację progiem.

In [ ]:
def compute_weighted_threshold(cord,arr,W):
    X, Y = cord
    arrx = arr[:,:,1] - X
    arry = arr[:,:,2] - Y
    out = arrx**2+arry**2
    arr_means= arr[:,:,0]
    return np.average(arr_means[out<=W**2],weights=out[out<=W**2])

    
def bin_int(img_org, W = 8):
    img = np.copy(img_org)
    (X, Y) = img.shape
    #thresholds and centers
    th_cen = []
    for x in range(0,X,W):
        temp = []
        for y in range(0,Y,W):
            frame = img[x:x+W,y:y+W]
            temp.append((np.mean(frame),x+int(W/2),y+int(W/2)))
        th_cen.append(temp)
    th_cen = np.array(th_cen)
    for x in range(X):
        for y in range(Y):
            img[x,y] = 255 if compute_weighted_threshold((x,y),th_cen,W)<img[x,y] else 0
    return img


In [ ]:
plt.imshow(bin_int(catalogue,16),'gray')